# Eigenvalue, Eigenvector
- - -

#### 이론

Covariance matrix $\mathbf{Q}$를 eigen decompostion을 수행하면 다음과 같은 형태로 분리가 됨:
\begin{align}
 \mathbf{Q} = \mathbf{U} \Lambda \mathbf{U}^T
\end{align}
- - -

1) $\mathbf{U}$의 column들이 eigenvector!

2) $\Lambda$의 대각 성분이 eigenvalue!

3) ```numpy``` 에서는 eigh 로 구함

#### numpy의 `linalg.eigh`를 이용한 Eigenvalue와 Eigenvector 구하기

<p><img alt="" src="https://drive.google.com/uc?id=1c-x_UUzVoT7KwcqiS8RKWP9H0kQXY1jq" style="height:400px; width:650px" /></p>


* **return 값 확인할 것 !!!** 

In [2]:
from numpy import linalg as LA
import numpy as np

testCov = [[3, 2, 1], [2, 3, 2], [1, 2, 3]]
eigval, eigvec = LA.eigh(testCov)

In [3]:
print('eigval의 type :', type(eigval))
print('eigval :', eigval)
print('\neigvec의 type :', type(eigvec))
print('eigvec :', eigvec)

eigval의 type : <class 'numpy.ndarray'>
eigval : [0.62771868 2.         6.37228132]

eigvec의 type : <class 'numpy.ndarray'>
eigvec : [[-4.54401349e-01  7.07106781e-01 -5.41774320e-01]
 [ 7.66184591e-01  5.55111512e-17 -6.42620551e-01]
 [-4.54401349e-01 -7.07106781e-01 -5.41774320e-01]]


1) 위 예제에서 ```eigval``` 은 $\Lambda$ 행렬의 대각성분만을 row vector 형태로 돌려줌

2) ```eigh``` 함수의 특징은 ```eigval```을 작은값에서 큰값 순으로 return함

3) **★★우리는 큰 값에서 작은 값으로 정렬(내림차순)하여 사용하기로 하고 수업시간에 배운것 관 같이 eigvec 순서도 정렬해줘야함★★**

예)
$$ \text{eigval} = [\lambda_1, \lambda_2, \lambda_3 ]$$
와
$$ \text{eigvec} = [\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3 ]$$
이 주어졌을때, 만약 $\lambda_3> \lambda_1> \lambda_2$ 순이라면, 
$$ \text{eigval} = [\lambda_3, \lambda_1, \lambda_2 ]$$

$$ \text{eigvec} = [\mathbf{u}_3, \mathbf{u}_1, \mathbf{u}_2 ]$$
로 재정렬

### Excercise 1 - 
#### Eigenvalue, eigenvector 순서 정렬하기 (20 point)

- - -

위 ```testCov```  **행렬의 eigval 과 eigvec**을 위 설명과 같이 `내림차순` 정렬하세요

<br>

**★tip! `numpy_array.argsort()` 필요하면 Google에 검색해 볼 것.!★**

```
# Exercise 1 output

- 정렬된 eigval: [6.37228132 2.         0.62771868]
- 정렬된 eigvec: [[-5.41774320e-01  7.07106781e-01 -4.54401349e-01]
 [-6.42620551e-01  5.55111512e-17  7.66184591e-01]
 [-5.41774320e-01 -7.07106781e-01 -4.54401349e-01]]
```

In [4]:
# 답작성
sort = np.argsort(eigval)[::-1]
eigval = eigval[sort]
eigvec = eigvec[:,sort]
# sort라는 변수에 numpy의 argsort함수를 사용해 내림 차순으로 정렬하는 것을 저장한다.
# 그런 후 eigval와 eigvec에 각각 대입시켜 내림차순으로 정렬 시킨다.

print("- 정렬된 eigval:", eigval)
print("- 정렬된 eigvec:", eigvec)

- 정렬된 eigval: [6.37228132 2.         0.62771868]
- 정렬된 eigvec: [[-5.41774320e-01  7.07106781e-01 -4.54401349e-01]
 [-6.42620551e-01  5.55111512e-17  7.66184591e-01]
 [-5.41774320e-01 -7.07106781e-01 -4.54401349e-01]]


## Computing PCA using RDDs

##  PCA

- - -

이번 실습에서는 PCA를 weather data 중에서 NY state의 SNWD data를 활용하여 수행함


1) Covariance를 구하는 부분은 코드를 아래 이미 작성된 코드를 따라가면 됨

2) 이미 지난 숙제를 통해서 배운 단계이니, 코드를 파악하는 것이 중요함

2) 단, 결과를 어떤 방식으로 저장하는지 파악을 해야 이후 단계에서 불러와서 처리할 수 있기 때문에 코드를 이해해야 합니다


In [5]:
from os import getcwd
getcwd()

'/home/jovyan/work'

In [6]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import *
import numpy as np

sc = SparkContext(master = 'local[*]')
sqlContext = SQLContext(sc)

In [ ]:
# weather parquet file download 하기
state='NY'
data_dir='Data/Weather'

tarname=state+'.tgz'
parquet=state+'.parquet'

%mkdir -p $data_dir
!rm -rf $data_dir/$tarname

from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1hAHV6vC6FvVgrYnoN-lR-IfH488-H121',
                                   dest_path = 'Data/Weather/NY.tgz')

!ls -lh $data_dir/$tarname

In [ ]:
# 다운받은 파일 압축 풀기
cur_dir,=!pwd
%cd $data_dir
!tar -xzf $tarname
!du ./$parquet
%cd $cur_dir

In [ ]:
# 압축 푼 파일 확인
parquet_path = data_dir+'/'+parquet
!du -sh $parquet_path

In [ ]:
# dataframe으로 변환 
df = sqlContext.read.parquet(parquet_path)
print(df.count())
df.show(5)

In [ ]:
# table 등록
sqlContext.registerDataFrameAsTable(df,'table')

In [ ]:
# 지난 주 숙제에 활용했던 함수 불러오기
import numpy as np
from numpy import linalg as LA

def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O = np.outer(X, X)
    N = 1 - np.isnan(O)
    return (O, N)

def sumWithNan(M1, M2):
    """Add two pairs of (matrix,count)"""
    (X1, N1) = M1
    (X2, N2) = M2
    N = N1 + N2
    X = np.nansum(np.dstack((X1,X2)), axis = 2)
    return (X, N)

# calculation function
def calc_func(S, N):
    # E is the sum of the vectors
    E = S[0, 1:]
    # NE is the number of not-nan antries for each coordinate of the vectors
    NE = np.float64(N[0, 1:])
    # Mean is the Mean vector (ignoring nans)
    Mean = E / NE
    # O is the sum of the outer products
    O = S[1:,1:]
    # NO is the number of non-nans in the outer product.
    NO = np.float64(N[1:,1:])
    return  E, NE, Mean, O, NO

In [ ]:
# Covaraince 연산하기 
# 마지막에 return 값 확인 필수~!!!!
def computeCov(RDDin):
    """computeCov recieves as input an RDD of np arrays, all of the same length, 
    and computes the covariance matrix for that set of vectors"""
    RDD = RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
    OuterRDD = RDD.map(outerProduct)   
    (S, N) = OuterRDD.reduce(sumWithNan)
    E, NE, Mean, O, NO = calc_func(S, N)

    Cov=O/NO - np.outer(Mean,Mean)
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    return {'E':E, 'NE':NE, 'O':O, 'NO':NO, 'Cov':Cov, 'Mean':Mean, 'Var':Var}

In [ ]:
# STAT_Description 본 함수 자료를 설명해주는 설명서임 "text"라고 간주하면됨

# description of data returned by computeOverAllDist
STAT_Descriptions = [
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
]



In [ ]:
# 위 함수를 활용한 covariance, mean 등을 연산하여 정리하는  main code

from time import time

sqlContext.registerDataFrameAsTable(df,'weather')

meas = 'SNWD'
STAT = {meas : {}}
Query = "SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
mdf = sqlContext.sql(Query)
print(meas,': shape of mdf is ', mdf.count())

data = mdf.rdd.map(lambda row: np.frombuffer(row['Values'],np.float16))
data.take(3)

# compute covariance matrix
OUT = computeCov(data)

In [ ]:
# OUT가 지난 숙제에서 구한 값들을 weather data 에 적용한 출력 결과임
# 저장 구조 확인 필!
# {'E':E, 'NE':NE, 'O':O, 'NO':NO, 'Cov':Cov, 'Mean':Mean, 'Var':Var}
print(OUT)

### Excercise 2 - 
#### 위에서 구한 결과에서 Covariance matrix 결과 출력 (5 point)

- - -

``OUT``를 활용하여 Covariance matrix 를 출력하세요.

<br>

```

# Exersize 1 output

 -> Covariance: [[24533.75726199 23941.70598575 23066.17130249 ...  7617.9044827
   7379.92048087  7567.8603963 ]
 [23941.70598575 25449.46142512 24666.0064621  ...  8210.2973947
   7965.20806728  8099.41512562]
 [23066.17130249 24666.0064621  26146.73215052 ...  8308.3103615
   7994.01160222  8034.86789953]
 ...
 [ 7617.9044827   8210.2973947   8308.3103615  ... 24550.3332284
  23487.93871963 22274.63652772]
 [ 7379.92048087  7965.20806728  7994.01160222 ... 23487.93871963
  24583.45550868 23206.83678396]
 [ 7567.8603963   8099.41512562  8034.86789953 ... 22274.63652772
  23206.83678396 24550.66736384]]
```

In [ ]:
### Excercise 2 - 
# 결과 출력

print("Covariance:", OUT['Cov']) 
# 딕셔녀리를 리턴하는 computeCov함수를 OUT에 저장했으며 covarianc를 구하기위해 OUT에 저장된 computeCov함수가 리턴하는 딕셔너리의 covariance
# 를 연산하는 부분을 사용하여 covariance를 계산한다.

### Excercise 3 - (10 point)

##### Eigenvector, Eigenvalue 구하기

- - -

위에서 구한 SWND data에 대한 Covariance 행렬을 활용하여 eigen value decompotion을 수행하세요. 

<br>

**task**

* 1.`eigen value 들은 eigval(변수명)` 로 저장 & `eigen vector 들은 eigvec(변수명)` 로 저장하세요

* 2. 순서를 eigen value 값이 큰값에서 작은 값이 되도록 정렬(내림차순)하고 eigen vector로 같은 원리로 정렬하세요

<br>

* 3. STAT dictionary에 'eigval'과 'eigvec' 항목을 추가하여 저장하는 부분은 주어지니, 형식에 맞춰서 저장하면 됩니다

<br> 

**★위 STAT 값은 아래에서 계속 활용되니 저장되는 형식을 숙지하세요★**


```
# task3-3 output
STAT[meas]['eigval'] [2742008.05793794  446193.78915167  224680.76680016  219381.26116906
  144138.98799991]

STAT[meas]['eigvec'] [[ 0.05931374  0.04804572 -0.11539879 -0.09368921 -0.0639553 ]
 [ 0.06106329  0.05172329 -0.11584723 -0.10096473 -0.07025214]
 [ 0.06355908  0.05964688 -0.12121571 -0.10394116 -0.08376024]
 [ 0.06565279  0.06796906 -0.12550461 -0.11100038 -0.0984372 ]
 [ 0.06690595  0.06990584 -0.130131   -0.10568974 -0.10882927]]

```

**task**

* 1.`eigen value 들은 eigval(변수명)` 로 저장 & `eigen vector 들은 eigvec(변수명)` 로 저장하세요(5 point)

* 2. 순서를 eigen value 값이 큰값에서 작은 값이 되도록 정렬(내림차순)하고 eigen vector로 같은 원리로 정렬하세요(5 point)

In [ ]:
eigval, eigvec = LA.eigh(OUT['Cov']) # eigen value와 eigen vector를 위에서 구한 OUT 딕셔너리에 저장된 Covariance행렬을 
                                     #각각 eigval과 eigvec으로 저장 한다.
sort = np.argsort(eigval)[::-1]     # 그런 후 내릴 차순으로 정렬 한다.
eigval = eigval[sort]
eigvec = eigvec[:,sort]


**task**

* 3. STAT dictionary에 'eigval'과 'eigvec' 항목을 추가하여 저장하는 부분은 주어지니, 형식에 맞춰서 저장하면 됩니다

```
# task3-3 output
STAT[meas]['eigval'] [2742008.05793794  446193.78915167  224680.76680016  219381.26116906
  144138.98799991]

STAT[meas]['eigvec'] [[ 0.05931374  0.04804572 -0.11539879 -0.09368921 -0.0639553 ]
 [ 0.06106329  0.05172329 -0.11584723 -0.10096473 -0.07025214]
 [ 0.06355908  0.05964688 -0.12121571 -0.10394116 -0.08376024]
 [ 0.06565279  0.06796906 -0.12550461 -0.11100038 -0.0984372 ]
 [ 0.06690595  0.06990584 -0.130131   -0.10568974 -0.10882927]]

```

In [ ]:
# collect all of the statistics in STAT[meas]
STAT[meas]['eigval'] = eigval
STAT[meas]['eigvec'] = eigvec
STAT[meas].update(OUT)

print("STAT[meas]['eigval']", STAT[meas]['eigval'][:5])
print("\nSTAT[meas]['eigvec']", STAT[meas]['eigvec'][:5,:5])

In [ ]:
# 새롭게 구한 eigval, eigvec dictionary에 잘 추가되었는지 확인
STAT['SNWD'].keys()

### Script for plotting yearly plots 

In [ ]:
from matplotlib import *
from matplotlib.pylab import subplot,axis
import sys
from pylab import *

### Visualization을 통한 분석

1. mean 및 mean $\pm$ std plot하기

아래 예제를 통해서 plot하는 방법을 숙지하세요.
```plot``` function은 아래와 같이 활용합니다:

```plot(x축값, y축값, color='b')```

여기서 ```color``` 는 graph의 색을 표현함 ```'b'```는 파란색, ```'r'```는 붉은색, ```'black'``` 검정 등

```figsize``` 는 전체 그래프의 크기표시하는것으로 아래 설정해둔 값을 활용하면 됨

```grid()``` 는 graph내에서 눈금자를 보이게하는 명령어임


In [ ]:
figure(figsize =(10,5)) # plot의 size 조절

yearday = [i for i in range(1,366)] # x축 설정(1일 ~ 365일)

plot(yearday,STAT[meas]['Mean'], color='r')
plot(yearday,STAT[meas]['Mean'] + np.sqrt(STAT[meas]['Var']), color='b')
plot(yearday,STAT[meas]['Mean'] - np.sqrt(STAT[meas]['Var']), color='g')
grid()

### Excercise 4 - (5 point)

날짜별로 기록한 data 수 plot하기 (날짜별 non-nan data 총수)

- - -

**task**

* ```STAT```에서 non-nan data 의 총 수를 저장한 부분(``NE``)을 활용하여 날짜별로 non-nan의 수를 출력하세요 

```
# Exercise 4 output
```
<p><img alt="" src="https://drive.google.com/uc?id=17y7GmCidHSPxk-jqpZ1KcFGBYWp4_oBt" style="height:300px; width:500px" /></p>

In [ ]:
# 답 작성
figure(figsize=(10,5))
yearday=[i for i in range(1,366)]
plot(yearday,STAT[meas]['NE'])
# 여기에 plot 을 활용하여 작성
# x축에는 1부터 366까지 반복하는 for문이 들어간 yearday만큼의 범위를 지정하여  meas딕셔너리 안의 딕셔너리의 non-nan data의 총수를 저장한 'NE'
# 를 y축으로 하여 그래프를 그린다.
grid()

### Excercise 5 - (20 point)

각 eigen vector가 전체 data 에 대해서 얼만큼 "설명" 해주는지 보여주는 그래프를 출력합니다.

- - -

각 eigen vector가 전체 데이터에 얼만큼 설명해주는지의 지표는 다음과 같이 구합니다

$$
i \text{ 번째 성분이 설명해주는 비율} = \frac{\lambda_i}{\sum_{i=1}^d \lambda_i}
$$

**task**

* 1. 다음의 절차에 따라 plot을 출력하세요(5 point)

    1) xaxis =  [0, 1, 2, 3, 4, 5]

    2) yaxis = 
$$ \left[ 0, 
\frac{\lambda_1}{\sum_{i=1}^d \lambda_i}, \frac{\lambda_1+\lambda_2}{\sum_{i=1}^d \lambda_i},\ldots, \frac{\lambda_1+\cdots + \lambda_5}{\sum_{i=1}^d \lambda_i} \right]$$

<br>

```
★yaxis에 대한 hint!!!★
# hint (cumsum 함수를 쓰면 편할수 있습니다)
# cumsum([a, b, c, d ]) output 은 [a, a+b, a+b+c, a+b+c+d]
print(cumsum(np.array([1, 2, 3, 4])))
---> [ 1  3  6 10]
```

<br>


* 2. 출력된 plot이 무엇을 의미하는지 설명을 간단하세 쓰세요(5 point)

- - -
```
# Exercise 5 output
```
<p><img alt="" src="https://drive.google.com/uc?id=1Dm7zTwGxR-GHUTzArSmzYVJNe1BEiK_b" style="height:350px; width:550px" /></p>

In [ ]:
# 작성
figure(figsize=(10,5))
xaxis = [0,1,2,3,4,5] # x축의 범위
yaxis = [0,(cumsum(STAT[meas]['eigval']))[0]/sum(STAT[meas]['eigval']),(cumsum(STAT[meas]['eigval']))[1]/sum(STAT[meas]['eigval']),
         (cumsum(STAT[meas]['eigval']))[2]/sum(STAT[meas]['eigval']),(cumsum(STAT[meas]['eigval']))[3]/sum(STAT[meas]['eigval']),
         (cumsum(STAT[meas]['eigval']))[4]/sum(STAT[meas]['eigval'])]
# eigval의 첫번째 인덱스 까지를 cumsum한 값을 eigval의 sum으로 나눈 값을 0다음의 연산값이고, 첫번째 인덱스부터 두번째 인덱스까지의 cumsum
# 한 값을 세번째 연산값이라 하면서 6번째 연산값인 eigval의 첫번째 인덱스부터 4번째 인덱스까지의 cumsum한 값에서 eigval의 sum한 값을 나눈
# 것을 6번째 연산값으로 하여 y축의 범위를 그린 후 그래프를 그린다.
plot(xaxis,yaxis)
grid()

# 설명 작성 (주석으로 설명)
# 1개의 eigen vector만 사용해서 60%의 복원력을 가짐을 볼 수 있고, 5개 정도를 사용하면 90% 가까지 복원력을 가지는 것을 볼 수 있다.

### Excercise 6 - (10 point)

Top 3 Eigen vector plot

- - -

**task**

* ```STAT```에서 Eigen vector를 저장한 부분(``eigvec``)을 활용하여 Top 3 Eigen vecotr의 plot 출력하세요 

```
# Exercise 6 output
```
<p><img alt="" src="https://drive.google.com/uc?id=1fd7kCcglsXZIR3RjMsi5njX9IfONewsW" style="height:400px; width:600px" /></p>

In [ ]:
# 답작성
figure(figsize=(10,5))
# eigvec의 상위 3개만을 인덱싱하여 그래프로 그린다. 첫행의 0번째 인덱스와 , 첫행의 1번째 인덱스, 첫행의 2번째 인덱스를 각각 빨강, 초록, 파랑 순으로 그래프를 그린다.
plot((STAT[meas]['eigvec'])[:,0], color = 'r') 
plot((STAT[meas]['eigvec'])[:,1], color = 'g')
plot((STAT[meas]['eigvec'])[:,2], color = 'b')
grid()


### Excercise 7 - (30 point)

Top 3 eigenvector로 데이터 복원하기.

- - -

SNWD 의 data vector를 $\mathbf{x}$라고 가정합니다. Eigenvector 3개로 estimate한 vector를 $\mathbf{w}_3$이라고 표기하면:

$$\mathbf{w}_3 = \bar{\mathbf{x}}+ \sum_{i=1}^3 c_i\mathbf{u}_i^T$$

로 표현됩니다. 여기서 $c_i$는

$$c_i =  (\mathbf{x}-\bar{\mathbf{x}})\cdot \mathbf{u}_i$$ 



입니다. $\mathbf{w}_3$와 $\mathbf{x}$ 벡터를 plot 하세요.

* 위 작업에서 활요할 data sample 은 주어집니다 (97번째 SWND 측정 Values np.array 입니다)

```x = np.array(data.collect()[96], dtype=np.float64)```

- - -

**task**

* 1. ``1,2,3 번째 eigenvector`` 와 ($\mathbf{x}-\bar{\mathbf{x}}$)를 내적(`np.dot`)한 $c_1$, $c_2$, $c_3$을 구하세요(5 point)

* 2. eigen vector 1,2,3번에 각각  $c_1$, $c_2$, $c_3$를 곱하고 (10 point)

* 3. 위에서 구한 모든 vector 들과  $\bar{\mathbf{x}}$ vector를 더합니다 (10 point)

* 4. 위에서 구한 $\mathbf{w}_3$와 $\mathbf{x}$를 plot 합니다 (10 point)

<br>

* 5. 96을 다른 값으로 바꿔서 놀아보세요~ 다른 sample에 대한 복원력을 보면, 잘되는 것도 있고 덜 복원되는 sample 도 있습니다 (문제는 아닙니다)

* 6. eigen vector의 수도 바꿔가면서 해볼 수 있습니다 (문제는 아닙니다)

* 7. 전체 365개를 활용하면 어떻게 될까요? (문제는 아닙니다)


```
# Exercise 7 output
```
<p><img alt="" src="https://drive.google.com/uc?id=1v-adyNo0QO5V_MjlvlD21t2CnvMaHFpI" style="height:400px; width:600px" /></p>

In [ ]:
x = np.array(data.collect()[96], dtype=np.float64)
c1 = np.dot(STAT[meas]['eigvec'][:,0],x-mean(x))
c2 = np.dot(STAT[meas]['eigvec'][:,1],x-mean(x))
c3 = np.dot(STAT[meas]['eigvec'][:,2],x-mean(x))
c4 = np.dot(STAT[meas]['eigvec'][:,3],x-mean(x))
c5 = np.dot(STAT[meas]['eigvec'][:,4],x-mean(x))
a = STAT[meas]['eigvec'][:,0] * c1
b = STAT[meas]['eigvec'][:,1] * c2
c = STAT[meas]['eigvec'][:,2] * c3
d = STAT[meas]['eigvec'][:,3] * c4
e = STAT[meas]['eigvec'][:,4] * c5
w3 = a+b+c+d+mean(x)
# eigvec의 상위 첫번째 부터 5번째 까지 해보았으며, x에서 x의 평균을 뺀 값과 내적한 후 그 값을 다시 eigvec의 상위 5개의 값에 곱한 후
# 그 값을 x의 평균치와 모두 더해준다. 원래 문제의 w를 3개 사용한 것 보다 5개로 늘려 주었더니 레이블 그래프를 더욱 더 세밀하게 따라가는 것을
# 볼 수 있었는데 이는 복원력이 더욱 더 좋아졌다고 볼 수 있다.

figure(figsize=(10,5))
plot(yearday, x, c='b')
plot(yearday, w3, c='r')
grid()